Questo notebook viene utilizzato per la lettura e analisi dei dati proveniente dal DB di WhoTeach

In [1]:
import pandas as pd

In [2]:
mdl_course = pd.read_csv("schemas/mdl_course.csv")
#,error_bad_lines=False)
mdl_course_categories = pd.read_csv("schemas/mdl_course_categories.csv")
mdl_course_sections = pd.read_csv("schemas/mdl_course_sections.csv")
full_text_section=  pd.read_csv("schemas/full_text_section.csv")
mdl_course_modules=  pd.read_csv("schemas/mdl_course_modules.csv")
mdl_url= pd.read_csv("schemas/mdl_url.csv")
mdl_metadata=pd.read_csv("schemas/mdl_metadata.csv")

FileNotFoundError: [Errno 2] File b'schemas/mdl_course.csv' does not exist: b'schemas/mdl_course.csv'

In [ ]:
#carico tutte le tabelle contenenti il nome delle risorse
#[1, 3, 4, 8, 9, 12, 13, 15, 16, 17, 20, 22]
#1
mdl_assign = pd.read_csv("schemas/mdl_assign.csv")
#3
mdl_book = pd.read_csv("schemas/mdl_book.csv")
#4
mdl_chat= pd.read_csv("schemas/mdl_chat.csv")
#8
mdl_folder = pd.read_csv("schemas/mdl_folder.csv")
#9
mdl_forum = pd.read_csv("schemas/mdl_forum.csv")
#12 
mdl_label = pd.read_csv("schemas/mdl_label.csv")
#13
mdl_lesson = pd.read_csv("schemas/mdl_lesson.csv")
#15
mdl_page = pd.read_csv("schemas/mdl_page.csv")
#16
mdl_quiz = pd.read_csv("schemas/mdl_quiz.csv")
#17
mdl_resource = pd.read_csv("schemas/mdl_resource.csv")
#20
mdl_url = pd.read_csv("schemas/mdl_url.csv")
#22
mdl_workshop= pd.read_csv("schemas/mdl_workshop.csv")

In [ ]:
#permettee di ottenere per ogni corso la sua categoria di appartenenza
mdl_course_categories.head(10)

In [ ]:
ris=mdl_course_categories.loc[mdl_course_categories['name'] == "Abilità informatiche"]
ris.name[0]

In [ ]:
#contiene l'informazione dei corsi effettivi
mdl_course.head()

In [ ]:
#permette di collegare il corso ad ogni suo modulo interno
mdl_course_sections.head()

In [ ]:
#informazione di ogni modulo
full_text_section.head()

In [ ]:
mdl_url.head()

In [ ]:
set(mdl_metadata.query("Property =='keywords' and Id_course == 217 and Id_course_sections==1673")["Id_resource"])

In [ ]:
#mdl_metadata.query("Property =='keywords' and Id_course == 217 and Id_course_sections==1673")

In [ ]:
statistica = mdl_course_categories.query("id ==22")
corsi=mdl_course.query("category == 22")
corso_id_217=corsi.query("id == 217")
moduli=mdl_course_sections.query("course ==217")
modulo=moduli.query("id == 1673 and section == 2")
risorse=mdl_course_modules.query("section==1673")
# 1. get keywords for each res
for res_id in set(risorse['id']):
    print(res_id)

# 2. get name

risorsa= risorse.query("instance == 839")
#accesso alla risorsa effettiva
#instance in mdl_course_modules indica l'id della risorsa nella tabella riferita dal campo module
risorsa_effettiva=mdl_url.query("id == 839")
list(risorsa_effettiva.name)[0]
risorsa

In [ ]:
# 1. get keywords for each res
a = []
for res_id in set(risorse['id']):
    print(res_id)
    a.append(mdl_metadata.loc[(mdl_metadata['Id_resource'] == res_id) & (mdl_metadata['Property'] == "keywords")]['Value'])
    break
    

In [3]:
database=[]
categories={}
courses={}
sections={}
name_section={}

In [5]:
for index in range(len(mdl_course_categories)):
    categories[mdl_course_categories.id[index]]=mdl_course_categories.name[index]
categories

NameError: name 'mdl_course_categories' is not defined

In [501]:
#tutti i corsi all'interno della piattaforma
#id	category	sortorder	fullname	shortname
for index in range(len(mdl_course)):
        ctgy=mdl_course.category[index]
        if ctgy != 0:
            #print(categories[ctgy])
            '''
            if mdl_course.shortname[index]=="'nan'":
                courses[mdl_course.id[index]]=[categories[ctgy],mdl_course.fullname[index],'null']
            else:
                courses[mdl_course.id[index]]=[categories[ctgy],mdl_course.fullname[index],mdl_course.shortname[index]]
            '''
            courses[mdl_course.id[index]]=[categories[ctgy],mdl_course.fullname[index]]
len(courses)

107

In [577]:
#tutte le sezioni di tutti i corsi
#id	course	section	name
for index in range(len(mdl_course_sections)):
    crse=mdl_course_sections.course[index]
    if crse != 1:
        sections[mdl_course_sections.id[index]]=[courses[crse][0],courses[crse][1]]

#len(full_text_section)
sections[1673]

['Probability and Statistics', 'Stochastic Estimation and Control']

In [ ]:
risorse_totali={}
for key in sections.keys():
    risorse_totali[key]=mdl_course_modules.query("section == "+str(key)) 

temp=[]
#id module instance id_section section/information
for key in risorse_totali.keys():
    for index,row in risorse_totali[key].iterrows():
        my_list=[int(row.id),int(row.module),int(row.instance),int(row.section)]
        temp.append(my_list)

risorse_informazione=[]
for i in temp:
    t=i+sections[i[3]]
    risorse_informazione.append(t)
risorse_informazione[0]



In [510]:
#tirare i nomi fuori da full_text_section poi inserirli nella posizione 2 in sections

for index in range(len(full_text_section)):
    name_section[full_text_section.id_section[index]]=full_text_section.name[index]

for key in sections.keys():
    if key in name_section.keys():
        sections[key].append(name_section[key])
        
len(sections)

646

In [490]:
#processo per ottenere la tabella dove si trova il nome della risorsa
#devo prendere l'ide da course_modules per poi prendere le keywords da ogni risorsa 
modulo_riga={}
moduli=[]
for index in range(len(mdl_course_modules)):
    modulo=mdl_course_modules.module[index]
    riga=mdl_course_modules.instance[index]
    ID=mdl_course_modules.id[index]
    if not(modulo in moduli):
        moduli.append(modulo)
        
    if mdl_course_modules.section[index] in sections.keys():
        modulo_riga[(modulo,riga,ID)]=sections[mdl_course_modules.section[index]]
moduli.sort()
moduli

[1, 3, 4, 8, 9, 12, 13, 15, 16, 17, 20, 22]

In [493]:
#elimino i moduli che non hanno nome
table_row={}
for key in modulo_riga.keys():
    if len(modulo_riga[key])>=3:
        table_row[key]=modulo_riga[key]
table_row

{(15, 10, 486): ['Abilità informatiche',
  'Editoria Digitale 2017/2018',
  "''Random Variables and Distributions''"],
 (12, 20, 487): ['Abilità informatiche',
  'Editoria Digitale 2017/2018',
  "''Random Variables and Distributions''"],
 (12, 21, 488): ['Abilità informatiche',
  'Editoria Digitale 2017/2018',
  "''Random Variables and Distributions''"],
 (12, 22, 489): ['Abilità informatiche',
  'Editoria Digitale 2017/2018',
  "''Random Variables and Distributions''"],
 (12, 23, 490): ['Abilità informatiche',
  'Editoria Digitale 2017/2018',
  "''Random Variables and Distributions''"],
 (12, 24, 491): ['Abilità informatiche',
  'Editoria Digitale 2017/2018',
  "''Random Variables and Distributions''"],
 (12, 25, 492): ['Abilità informatiche',
  'Editoria Digitale 2017/2018',
  "''Random Variables and Distributions''"],
 (12, 26, 493): ['Abilità informatiche',
  'Editoria Digitale 2017/2018',
  "''Random Variables and Distributions''"],
 (12, 27, 494): ['Abilità informatiche',
  'Edit

In [474]:
#processo per ottenere il nome di risorsa , il suo moodulo , corso e categoria7
resources=[]
for key in table_row.keys():
    table=key[0]
    row=key[1]
    if table==1:
        a=list(mdl_assign.query("id=="+str(row)).name)[0]
        
        print(a,table_row[(table,row)][0:3])
    '''
    elif table ==3:
        a=list(mdl_book.query("id=="+str(row)).name)[0]
        table_row[(table,row)].append(a)
    elif table ==4:
        a=list(mdl_chat.query("id=="+str(row)).name)[0]
        table_row[(table,row)].append(a)
    elif table ==8:
        a=list(mdl_folder.query("id=="+str(row)).name)[0]
        table_row[(table,row)].append(a)
    elif table ==9:
        a=list(mdl_forum.query("id=="+str(row)).name)[0]
        table_row[(table,row)].append(a)
    elif table ==12:
        a=list(mdl_label.query("id=="+str(row)).name)[0]
        table_row[(table,row)].append(a)
    elif table ==13:
        a=list(mdl_lesson.query("id=="+str(row)).name)[0]
        table_row[(table,row)].append(a)
    elif table ==15:
        a=list(mdl_page.query("id=="+str(row)).name)[0]
        table_row[(table,row)].append(a)
    elif table ==16:
        a=list(mdl_quiz.query("id=="+str(row)).name)[0]
        table_row[(table,row)].append(a)
    elif table ==17:
        a=list(mdl_resource.query("id=="+str(row)).name)[0]
        table_row[(table,row)].append(a)
    elif table ==20:
        a=list(mdl_url.query("id=="+str(row)).name)[0]
        table_row[(table,row)].append(a)
    elif table ==22:
        a=list(mdl_workshop.query("id=="+str(row)).name)[0]
        table_row[(table,row)].append(a)
    else: 
        print('error')
    '''

resources

Test ['Abilità informatiche', 'ITS Rizzoli - Programmatore Java, Android e Web', "''Analysis of variance, with discussion of Bayesian and frequentist statistics''"]
CREA IL TUO SITO  ['Abilità informatiche', 'ITS Rizzoli -  Multichannel', "''Case study: tungsten CVD DOE/RSM''"]


[]

In [448]:
#processo per inserire la lista di keywords per ogni risorsa
for key in table_row.keys():
    print(table_row[key])

['Abilità informatiche', 'Editoria Digitale 2017/2018', "''Random Variables and Distributions''", 'Risorse online', 'Lezioni05/10/2017Presentazione del corso, modalità...', '10/10/2017Scenari della nuova editoria&nbsp;Lettur...', '12/10/2017Il libro digitale e le sue origini&nbsp;...', '17/10/2017Il mercato del libro&nbsp;Letture:&nbsp;...', "19/10/2017Il mercato dell'editoria (lucidi precede...", '24/10/2017Il mercato del libro digitale', '02/11/2017La lettura che cambia (lucidi precedenti...', '07/11/2017Il formato ePub 2', '09/11/2017Delucidazione sulle richieste del proget...', '14/11/2017Implementazione della struttura di un do...', '16/11/2017Il formato ePub 3', '21/11/2017Le risorse multimediali in ePub 3', '23/11/2017Gli overlay audio in ePub 3. Esempio: Il...', '28/11/2017lezione sospesa per test nazionale di Me...', '16/01/2018Test di valutazione per tutti gli appell...']
['Abilità informatiche', 'Editoria Digitale 2017/2018', "''Random Variables and Distributions''", 'Risors

In [420]:
#associare il rating medio per ogni corso ad ogni risorsa

In [ ]:
#realizzare analisi dei dati 